### Objectif:

Le but de ce notebook est d'entraîner un modèle de machine learning pour remplir les valeurs manquantes de la variable <mark>ROOM_COUNT</mark>, en fait nous allons considérer un problème supervisé avec <mark>ROOM_COUNT</mark> comme variable cible.
- Nous avons commencé par construire notre base de données, qui se compose de 5 variables explicatives et de la variable cible.
- Nous avons divisé notre base de données en données de test, les lignes avec des valeurs manquantes de la variable cible, et le reste de la base de données, qui consiste pour l'essentiel en notre base de données d'entrainnement et de validation.
- Après avoir construit et validé notre modèle (Random Forest), nous avons prédit les valeurs manquantes de la variable cible avec ce dérnier.

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [2]:
#import data

data = pd.read_csv('..\data\sub_data.csv')


# Preparation of data
target = 'ROOM_COUNT' # Variable cible qu'on veut remplir à la fin
model_var = ['PROPERTY_TYPE', 'NEW_BUILD', 'SURFACE', 'PRICE', 'EXCLUSIVE_MANDATE', 'MARKETING_TYPE'] # nos variables que nous avons choisi
model_data = data[model_var + [target]]

In [3]:
# Fonction pour supprimer les lignes avec des valeurs manquantes

def drop_rows_with_missing_values(data, columns_to_check):
    
    missing_values = data[columns_to_check].isnull().any(axis=1)

    data_cleaned = data[~missing_values]

    return data_cleaned

In [4]:
model_data = drop_rows_with_missing_values(model_data, model_var)

In [5]:
# model_data.dropna(subset=['NEW_BUILD', 'SURFACE', 'PRICE'], inplace=True)

In [6]:
model_data.isna().sum()

PROPERTY_TYPE          0
NEW_BUILD              0
SURFACE                0
PRICE                  0
EXCLUSIVE_MANDATE      0
MARKETING_TYPE         0
ROOM_COUNT           325
dtype: int64

In [7]:
# Dans cette section, nous essayons d'encoder nos variables catégorielles.

In [8]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
enc.fit(model_data[["PROPERTY_TYPE"]])
model_data[["PROPERTY_TYPE"]] = enc.transform(model_data[["PROPERTY_TYPE"]])

In [9]:

model_data["EXCLUSIVE_MANDATE"] = model_data["EXCLUSIVE_MANDATE"].astype(int)

In [10]:
model_data['NEW_BUILD'] = model_data['NEW_BUILD'].map({'True': 1, 'False': 0, '-1': -1})

In [11]:
model_data['MARKETING_TYPE'] = model_data['MARKETING_TYPE'].map({'RENT': 1, 'SALE': 0})

#### Notre dataset resemble à cela : 

In [12]:
model_data.head()

,PROPERTY_TYPE,NEW_BUILD,SURFACE,PRICE,EXCLUSIVE_MANDATE,MARKETING_TYPE,ROOM_COUNT
0,0.0,0,72.0,1979.0,0,1,3.0
1,0.0,0,48.0,429000.0,0,0,2.0
2,0.0,0,267.0,4970000.0,0,0,6.0
3,4.0,0,50.0,89.0,0,1,0.0
4,2.0,0,330.0,2790000.0,0,0,8.0


In [13]:
# Nous allons essayer d'extraire la base test avec les valeurs manquantes, et la base d'entr et de validation sans valeurs manquantes.

data_with_missing = model_data[model_data[target].isna()]
data_without_missing = model_data.dropna(subset=[target])

In [14]:
X = data_without_missing.drop(columns=[target])
y = data_without_missing[target]

In [15]:
# diviser X et y en ensembles de training et testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
X.isna().sum()

PROPERTY_TYPE        0
NEW_BUILD            0
SURFACE              0
PRICE                0
EXCLUSIVE_MANDATE    0
MARKETING_TYPE       0
dtype: int64

In [17]:
# Model 

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=123, verbose=0, warm_start=False)

clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=123)

In [18]:
y_pred_test = clf.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

Model accuracy score: 0.9344


In [20]:
from sklearn.metrics import balanced_accuracy_score

print('Model balanced accuracy score: {0:0.4f}'. format(balanced_accuracy_score(y_test, y_pred_test)))

Model balanced accuracy score: 0.8532


#### Nous avons essayé de travailler avec une balanced accuracy parce qu'il y a un déséquilibre dans notre variable cible.
#### Comme nous pouvons le constater, notre accuracy est très bonne. (85%)

In [21]:
# Prédire les valeurs cibles manquantes
X_with_missing = data_with_missing.drop(columns=[target])
predicted_values = clf.predict(X_with_missing)

In [22]:
data_with_missing[target] = predicted_values

final_data = pd.concat([data_with_missing, data_without_missing])

In [23]:
final_data

,PROPERTY_TYPE,NEW_BUILD,SURFACE,PRICE,EXCLUSIVE_MANDATE,MARKETING_TYPE,ROOM_COUNT
7,3.0,-1,16.00,60.0,0,1,1.0
8,3.0,-1,11.00,100.0,0,1,1.0
10,4.0,0,95.00,64500.0,0,0,0.0
17,4.0,0,1105.00,25783.0,0,1,0.0
18,3.0,-1,12.74,20000.0,0,0,1.0
...,...,...,...,...,...,...,...
2158,0.0,0,121.70,1250000.0,0,0,5.0
2159,0.0,0,50.00,465000.0,0,0,2.0
2160,0.0,0,31.00,1010.0,0,1,2.0
2161,0.0,0,67.00,719000.0,0,0,3.0


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Remplir les valeur manquantes de ROOM_COUNT

In [24]:
# Localiser et remplir
data.loc[X_with_missing.index, "ROOM_COUNT"] = predicted_values

In [25]:
## enregistrer les données dans un fichier 'new_data.csv'
# data.to_csv('..\data\new_data.csv')

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------